# Extraction : updating

## Packages installation and imports

In [6]:
#!pip install python-dotenv
#!pip install requests_oauthlib
!pip install numpy
#!pip install cython
!pip install fastparquet
#!pip install pyarrow
!pip install bs4
!pip install requests
!pip install selenium
!pip install pandas
!pip install Unidecode
!pip install unicodedata2

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import fastparquet as fp
import unicodedata
import time
import datetime

### 1) Get last date record

In [18]:
df = pd.read_parquet("France/data_fr_origin_prepared.parquet")

### Set a 1st layer extraction: last update => today included

In [19]:
#Get date of last records
most_recent_date = pd.to_datetime(df['journal_dateType']).max() # convert the column to datetime

#split this date into day, month, year (to use it in scraping url)
starting_update_on = most_recent_date + datetime.timedelta(days=1)
starting_day = starting_update_on.day
strating_month = starting_update_on.month
starting_year = starting_update_on.year

#end date
end = datetime.date.today()
end_day = end.day
end_month = end.month
end_year = end.year

print("from " + str(starting_update_on) +" to " + str(end))

from 2023-03-18 00:00:00 to 2023-06-12


In [20]:
url = 'https://www.legifrance.gouv.fr/jorf/jo/period/?datePubli='+ str(starting_day) + '%2F' + str(strating_month) +'%2F' + str(starting_year) + '%3E+' + str(end_day) +'%2F' + str(end_month) +'%2F' + str(end_year) +'&page=1&pageSize=100'
BASE_URL = "https://www.legifrance.gouv.fr"
#Open browser
def openDriver():
    # Set up the driver and open the website
    service = Service('abfss://storagecontainerbis@storageaccountbis.dfs.core.windows.net/data/chrome.exe')  # Replace with the path to ChromeDriver executable
    driver = webdriver.Chrome(service=service)
    
    return driver

def getPageContent(url, driver):
    driver.get(url)
    # Wait for the page to load and get the HTML content
    wait = WebDriverWait(driver, 0.2)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h2')))
    html = driver.page_source
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    return soup

# Find nbr of pages of the research:
def getNbrOfPages(soup):
    # Find the ul pager in the HTML content
    ul_pager_list = soup.find('ul', class_='pager type-ajax')
    if not ul_pager_list:
        ul_pager_list = soup.find('ul', class_='pager type-reload')
        li_list = ul_pager_list.find_all('li')
        if(len(li_list)>1):
            #Get the second last li tag in the ul and find number of pages
            #(the last element being the 'suivant' button)
            last_li = ul_pager_list.find_all('li')[-2]
        else:
            last_li = ul_pager_list.find_all('li')[-1]
        nbr_of_pages = re.search(r'(\d+)', last_li.text.strip()).group(1)
    
    else:
        last_li = ul_pager_list.find_all('li')[-2]
        # Get the content of the last li
        nbr_of_pages = re.search(r'(\d+)', last_li.text.strip()).group(1)
        
    return int(nbr_of_pages)

def getNbrOfResults(soup):
    # Find <p> where nbr of results is stored
    str_results_line = soup.find('p', class_ = 'nb-result').text.strip()
    match_obj = re.search(r'^(\d+)', str_results_line) #find match object of the regex: 1st occurence of digit in the string
    nbr_results = match_obj.group(1) # get value of the match.
    return int(nbr_results)

# Get dates and links from one page (argument: page nummer)
def getHappeningsFromOnePage(url_indexed, driver):
    happenings_data = []
    soup = getPageContent(url_indexed, driver)
    
    #Loop on all articles in results
    articles = soup.find_all('article')
    for art in articles:
        happening_date = art.find("h2", class_="title-result-item").text.strip()
        #print(happening_date)
        div_content = art.find("div", class_="content")
        li_tags = div_content.find("ul").find_all("li")
        happening_summary_link = BASE_URL + li_tags[0].find('a')['href']
        if(len(li_tags)>1):
            happening_summary_pdf_link = li_tags[1].find('a')['href']
            if(happening_summary_pdf_link == 'javascript:void(0)'):
                happening_summary_pdf_link = "link protected"
            else:
                happening_summary_pdf_link = BASE_URL+"{0}".format(happening_summary_pdf_link)
        hpng_data = [happening_date, happening_summary_link, happening_summary_pdf_link]
        
        happenings_data.append(hpng_data)    
    
    return happenings_data        
        
def scrapingFlow(url):
    driver = openDriver()
    #get 1st page of research to get nbr_pages
    soup = getPageContent(url, driver)
    nbr_pages = getNbrOfPages(soup)
    nbr_results = getNbrOfResults(soup)
    print(str(nbr_pages) + 'pages.')
    print(str(nbr_results) + 'results.')
    happenings = []
    #limited search to first 10k result.
    #if(nbr_pages > 99):
    #    nbr_pages = 99
    
    for idx in range(1,nbr_pages+1):
        #adapt the url
        print('scanning page n°: '+str(idx))
        url_begin = 'https://www.legifrance.gouv.fr/jorf/jo/period/?datePubli='+ str(starting_day) + '%2F' + str(strating_month) +'%2F' + str(starting_year) + '%3E+' + str(end_day) +'%2F' + str(end_month) +'%2F' + str(end_year) +'&page='
        url_end = '&pageSize=100'
        url_indexed = url_begin + str(idx) + url_end
        
        if(idx<=99):
            #links.extend(getDataFromOnePage(url_indexed, driver))
            res = getHappeningsFromOnePage(url_indexed, driver)
            #print(res)
            happenings.extend(res)
    driver.quit()
    #return links
    return happenings

#App
data_scraped = scrapingFlow(url)

1pages.
70results.
scanning page n°: 1


In [23]:
print(end)

#turn results list into a pandas dataframe
df1 = pd.DataFrame(data_scraped,columns =['journal_date', 'journal_link', 'journal_pdf_link'])
file_name = 'happenings_from_'+str(starting_update_on.year) +'-'+ str(starting_update_on.month) + '-' + str(starting_update_on.day) + '_to_'+str(end)+'.parquet'
print(file_name)
#store to .parquet
df1.to_parquet(file_name)

pd.read_parquet(file_name)

2023-06-12
happenings_from_2023-3-18_to_2023-06-12.parquet


,journal_date,journal_link,journal_pdf_link
0,11 juin 2023,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...
1,10 juin 2023,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...
2,9 juin 2023,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...
3,8 juin 2023,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...
4,7 juin 2023,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...
...,...,...,...
64,23 mars 2023,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...
65,22 mars 2023,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...
66,21 mars 2023,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...
67,19 mars 2023,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...


### Set 2nd layer extraction

In [24]:
df1 = pd.read_parquet(file_name)
JO_list = df1.values.tolist()
print(len(JO_list))
print(JO_list[8])

69
['2 juin 2023', 'https://www.legifrance.gouv.fr/jorf/jo/id/JORFCONT000047618565', 'https://www.legifrance.gouv.fr/download/pdf?id=qxDQLSIvw6U21Ulv6QiLmKBVOO4Ees1U922iCwIn4_8=']


In [25]:
def getPageContent(url, driver):
    driver.get(url)
    # Wait for the page to load and get the HTML content
    wait = WebDriverWait(driver, 0.2)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'div')))
    html = driver.page_source
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def scrapingFlow2(url):
    ids_titles_links = []
    #driver = openDriver()
    soup = getPageContent(url, driver)
    div_content = soup.find('div', class_="content-page")
    ol_tags = div_content.find_all('ol')
    for ol_tag in ol_tags:
        li_tags = ol_tag.find_all('li')
        for li_tag in li_tags:
            #print(li_tag)
            id_ = li_tag['id']
            title = li_tag.find('a').text
            #print(title)
            link = BASE_URL + li_tag.find('a')['href']
            ids_titles_links.append([id_, title, link])
    #print(ids_titles_links)
    
    return ids_titles_links


driver = openDriver()
#data_2nd_layer = JO_list
data_gathered = []
#df2 = pd.DataFrame 
for row_idx in range(0, len(df1['journal_link'])): 
#for row_idx in range(0, 5):
#for idx in range(1):    
    #return a list of articles (id, title, link)
    print("idx:" + str(row_idx))
    articles = scrapingFlow2(JO_list[row_idx][1])
    for art in articles:
        date = JO_list[row_idx][0]
        happenings_link = JO_list[row_idx][1]
        happenings_pdf_link = JO_list[row_idx][2]
        art_id = art[0]
        art_title = art[1]
        art_link = art[2]
        row = [date,happenings_link,happenings_pdf_link, art_id, art_title, art_link]
        data_gathered.append(row)
        #new_row = {'Journal_date': date, 'happenings_link':happenings_link , 'happenings_pdf_link':happenings_pdf_link, 'article_id':art_id, 'article_title': art_title, 'article_link':art_link}
    print("JO-DATE:  " + date)
    #print(len(data_gathered))
    #data_2nd_layer[row_idx].append(scrapingFlow2(data_2nd_layer[row_idx][1]))
driver.close()

idx:0
JO-DATE:  11 juin 2023
idx:1
JO-DATE:  10 juin 2023
idx:2
JO-DATE:  9 juin 2023
idx:3
JO-DATE:  8 juin 2023
idx:4
JO-DATE:  7 juin 2023
idx:5
JO-DATE:  6 juin 2023
idx:6
JO-DATE:  4 juin 2023
idx:7
JO-DATE:  3 juin 2023
idx:8
JO-DATE:  2 juin 2023
idx:9
JO-DATE:  1 juin 2023
idx:10
JO-DATE:  31 mai 2023
idx:11
JO-DATE:  28 mai 2023
idx:12
JO-DATE:  27 mai 2023
idx:13
JO-DATE:  26 mai 2023
idx:14
JO-DATE:  25 mai 2023
idx:15
JO-DATE:  24 mai 2023
idx:16
JO-DATE:  23 mai 2023
idx:17
JO-DATE:  21 mai 2023
idx:18
JO-DATE:  20 mai 2023
idx:19
JO-DATE:  18 mai 2023
idx:20
JO-DATE:  17 mai 2023
idx:21
JO-DATE:  16 mai 2023
idx:22
JO-DATE:  14 mai 2023
idx:23
JO-DATE:  13 mai 2023
idx:24
JO-DATE:  12 mai 2023
idx:25
JO-DATE:  11 mai 2023
idx:26
JO-DATE:  10 mai 2023
idx:27
JO-DATE:  7 mai 2023
idx:28
JO-DATE:  6 mai 2023
idx:29
JO-DATE:  5 mai 2023
idx:30
JO-DATE:  4 mai 2023
idx:31
JO-DATE:  3 mai 2023
idx:32
JO-DATE:  30 avril 2023
idx:33
JO-DATE:  29 avril 2023
idx:34
JO-DATE:  28 avr

In [26]:
print(len(data_gathered))
print(data_gathered[0])

6512
['11 juin 2023', 'https://www.legifrance.gouv.fr/jorf/jo/id/JORFCONT000047664395', 'https://www.legifrance.gouv.fr/download/pdf?id=kONM7XCFtUP-8qj-5T_gzZ9zMW9r0VCLrkV8AmAAT3o=', 'JORFTEXT000047664400', "Arrêté du 25 mai 2023 portant approbation d'une prise de participation financière par l'Agence française de développement", 'https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000047664400']


In [27]:
#turn results list into a pandas dataframe
df = pd.DataFrame(data_gathered,columns =['journal_date', 'journal_link', 'journal_pdf_link', 'article_id', 'article_title', 'article_link'])

#store to .parquet (erase the previous file version)
df.to_parquet(file_name)

### Set 3nd layer extraction (content of articles)

In [28]:
data_2 = pd.read_parquet(file_name, engine='fastparquet')

In [29]:
docs = []
#datas_per_page = []
service = Service('path/to/chromedriver')
driver = webdriver.Chrome(service=service)
BASE_URL = 'https://www.legifrance.gouv.fr/jorf/id/'

def cleanPatterns(txt):
    regex_pattenrs_to_clean = "(\\[a-z]|Masquer les articles et les sections abrogés|Liens relatifs|Replier|Déplier|Section [0-9]* : |Sous section [0-9]*|Sous-section [0-9]*|\(Article [A-Z][0-9]*-[0-9]*\)| \(Articles [A-Z][0-9]*-[0-9]* à [A-Z][0-9]*-[0-9]*\)| Article [0-9]+ : |Versions|Article [0-9]+)"
    cleaned_txt = re.sub(r""+regex_pattenrs_to_clean, "", txt, flags=re.IGNORECASE)
    #print(result)
    
    return cleaned_txt

#loop inside the dataframe from 10k_data.parquet
#for row_idx in range(1267, len(data['doc_id'])):    
#for row_idx in range(500, 1000):
#for row_idx in range(402, 410):
#for row_idx in range(2265, 2280):
#for row_idx in range(0, len(data['article_id'])):
for row_idx in range(0, len(data_2['article_id'])):
#for row_idx in range(250000, 500000): 
    datas_per_page = []
    if(row_idx % 150 == 0):
        driver.close()
        time.sleep(1) # give selenium a sleep time to prevent the target server to block our requests (error 400)
        driver = webdriver.Chrome(service=service)
    
    try:
        print("row:" + str(row_idx))
        article_id = data_2['article_id'].loc[data_2.index[row_idx]]
        print("article_id: " + article_id)
        url = "https://www.legifrance.gouv.fr/jorf/id/" + article_id   
        driver.get(url)
        wait = WebDriverWait(driver, 5)
        wait.until(EC.presence_of_element_located((By.ID, 'liste-sommaire')))
        html = driver.page_source
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        
        child_articles = soup.find_all('article')
        for art in child_articles:
            div_content_tag = art.find('p', {'class':'name-article'})
            if div_content_tag :
                if div_content_tag.find('a'):
                    child_title = div_content_tag.find('a').text.strip()
                    #print("child title: "+ child_title)
                    child_link = 'https://www.legifrance.gouv.fr' + div_content_tag.find('a').get('href')
                    #print("child link: " + child_link)
                else:
                    child_title = ''
                    child_link = ''
                child_text = art.find('div', {'class':'content'})
                if(child_text):
                    child_text = child_text.text.strip()
                    child_text = unicodedata.normalize("NFKC", child_text)
                    child_text = cleanPatterns(child_text)
                    #print("child text: " + child_text)
                else:
                    child_title = 'No Title (no child article)'
                    child_link = 'No Link (no child article)'
                    child_text = 'No Text (no child text)'

                #data = Data(doc_id, title, link, inner_title, inner_link, teaser)
                data = [article_id, child_title, child_link, child_text]
                #print("Data : "+ str(data))
                datas_per_page.append(data)

        
    except: #case: if hmtl does not contain <ul id="liste-sommaire">
        print("error: no text")
        #text = "BLANK"
    finally:    
        docs.extend(datas_per_page) 
        #print(docs)
        
        
        
df3 = pd.DataFrame(docs,columns =['article_id', 'child_title','child_link','child_text'])
#df3.to_parquet('data_from_2000_to_now_text.parquet')
#df.to_parquet('test_data_text.parquet') 

row:0
article_id: JORFTEXT000047664400
row:1
article_id: JORFTEXT000047664405
row:2
article_id: JORFTEXT000047664410
row:3
article_id: JORFTEXT000047664424
row:4
article_id: JORFTEXT000047664451
row:5
article_id: JORFTEXT000047664454
row:6
article_id: JORFTEXT000047664460
row:7
article_id: JORFTEXT000047664469
row:8
article_id: JORFTEXT000047664486
row:9
article_id: JORFTEXT000047664496
row:10
article_id: JORFTEXT000047664498
row:11
article_id: JORFTEXT000047664526
row:12
article_id: JORFTEXT000047664541
row:13
article_id: JORFTEXT000047664611
row:14
article_id: JORFTEXT000047664631
row:15
article_id: JORFTEXT000047664639
row:16
article_id: JORFTEXT000047664643
row:17
article_id: JORFTEXT000047664651
row:18
article_id: JORFTEXT000047664664
row:19
article_id: JORFTEXT000047664759
row:20
article_id: JORFTEXT000047664777
row:21
article_id: JORFTEXT000047664779
row:22
article_id: JORFTEXT000047664782
row:23
article_id: JORFTEXT000047664784
row:24
article_id: JORFTEXT000047664786
row:25
art

row:203
article_id: JORFTEXT000047657813
row:204
article_id: JORFTEXT000047657815
row:205
article_id: JORFTEXT000047657817
row:206
article_id: JORFTEXT000047657821
row:207
article_id: JORFTEXT000047657824
row:208
article_id: JORFTEXT000047657826
row:209
article_id: JORFTEXT000047657828
row:210
article_id: JORFTEXT000047657830
row:211
article_id: JORFTEXT000047657833
row:212
article_id: JORFTEXT000047657838
row:213
article_id: JORFTEXT000047657848
row:214
article_id: JORFTEXT000047657853
row:215
article_id: JORFTEXT000047650221
row:216
article_id: JORFTEXT000047650227
row:217
article_id: JORFTEXT000047650234
row:218
article_id: JORFTEXT000047650243
row:219
article_id: JORFTEXT000047650251
row:220
article_id: JORFTEXT000047650267
row:221
article_id: JORFTEXT000047650286
row:222
article_id: JORFTEXT000047650323
row:223
article_id: JORFTEXT000047650325
row:224
article_id: JORFTEXT000047650328
row:225
article_id: JORFTEXT000047650334
row:226
article_id: JORFTEXT000047650339
row:227
article_

row:403
article_id: JORFTEXT000047641415
row:404
article_id: JORFTEXT000047641433
row:405
article_id: JORFTEXT000047641441
row:406
article_id: JORFTEXT000047641451
row:407
article_id: JORFTEXT000047641455
row:408
article_id: JORFTEXT000047641457
row:409
article_id: JORFTEXT000047641459
row:410
article_id: JORFTEXT000047641461
row:411
article_id: JORFTEXT000047641464
row:412
article_id: JORFTEXT000047641466
row:413
article_id: JORFTEXT000047641468
row:414
article_id: JORFTEXT000047641470
row:415
article_id: JORFTEXT000047641472
row:416
article_id: JORFTEXT000047641474
row:417
article_id: JORFTEXT000047641477
row:418
article_id: JORFTEXT000047641482
row:419
article_id: JORFTEXT000047641495
row:420
article_id: JORFTEXT000047641508
row:421
article_id: JORFTEXT000047641521
row:422
article_id: JORFTEXT000047641534
row:423
article_id: JORFTEXT000047641548
row:424
article_id: JORFTEXT000047641558
row:425
article_id: JORFTEXT000047641578
row:426
article_id: JORFTEXT000047641585
row:427
article_

row:603
article_id: JORFTEXT000047624201
row:604
article_id: JORFTEXT000047624262
row:605
article_id: JORFTEXT000047624272
row:606
article_id: JORFTEXT000047624292
row:607
article_id: JORFTEXT000047624306
row:608
article_id: JORFTEXT000047624314
row:609
article_id: JORFTEXT000047624316
row:610
article_id: JORFTEXT000047624322
row:611
article_id: JORFTEXT000047624328
row:612
article_id: JORFTEXT000047624334
row:613
article_id: JORFTEXT000047624339
row:614
article_id: JORFTEXT000047624341
row:615
article_id: JORFTEXT000047624343
row:616
article_id: JORFTEXT000047624345
row:617
article_id: JORFTEXT000047624347
row:618
article_id: JORFTEXT000047624350
row:619
article_id: JORFTEXT000047624352
row:620
article_id: JORFTEXT000047624354
row:621
article_id: JORFTEXT000047624356
row:622
article_id: JORFTEXT000047624358
row:623
article_id: JORFTEXT000047624361
row:624
article_id: JORFTEXT000047624363
row:625
article_id: JORFTEXT000047624365
row:626
article_id: JORFTEXT000047624368
row:627
article_

row:803
article_id: JORFTEXT000047614378
row:804
article_id: JORFTEXT000047614380
row:805
article_id: JORFTEXT000047614382
row:806
article_id: JORFTEXT000047614384
row:807
article_id: JORFTEXT000047614389
row:808
article_id: JORFTEXT000047614392
row:809
article_id: JORFTEXT000047614394
row:810
article_id: JORFTEXT000047614396
row:811
article_id: JORFTEXT000047614398
row:812
article_id: JORFTEXT000047614400
row:813
article_id: JORFTEXT000047614402
row:814
article_id: JORFTEXT000047614404
row:815
article_id: JORFTEXT000047614406
row:816
article_id: JORFTEXT000047614408
row:817
article_id: JORFTEXT000047614410
row:818
article_id: JORFTEXT000047614412
row:819
article_id: JORFTEXT000047614414
row:820
article_id: JORFTEXT000047614416
row:821
article_id: JORFTEXT000047614418
row:822
article_id: JORFTEXT000047614420
row:823
article_id: JORFTEXT000047614422
row:824
article_id: JORFTEXT000047614424
row:825
article_id: JORFTEXT000047614426
row:826
article_id: JORFTEXT000047614428
row:827
article_

...

row:5706
article_id: JORFTEXT000047358952
row:5707
article_id: JORFTEXT000047358962
row:5708
article_id: JORFTEXT000047358968
row:5709
article_id: JORFTEXT000047358974
row:5710
article_id: JORFTEXT000047358976
row:5711
article_id: JORFTEXT000047358979
row:5712
article_id: JORFTEXT000047358990
row:5713
article_id: JORFTEXT000047358992
row:5714
article_id: JORFTEXT000047358994
row:5715
article_id: JORFTEXT000047358996
row:5716
article_id: JORFTEXT000047358998
row:5717
article_id: JORFTEXT000047359001
row:5718
article_id: JORFTEXT000047359003
row:5719
article_id: JORFTEXT000047359005
row:5720
article_id: JORFTEXT000047359008
row:5721
article_id: JORFTEXT000047359013
row:5722
article_id: JORFTEXT000047359032
row:5723
article_id: JORFTEXT000047359059
row:5724
article_id: JORFTEXT000047359079
row:5725
article_id: JORFTEXT000047359097
row:5726
article_id: JORFTEXT000047359106
row:5727
article_id: JORFTEXT000047359110
row:5728
article_id: JORFTEXT000047359117
row:5729
article_id: JORFTEXT00004

row:5902
article_id: JORFTEXT000047340895
row:5903
article_id: JORFTEXT000047340902
row:5904
article_id: JORFTEXT000047340915
row:5905
article_id: JORFTEXT000047340924
row:5906
article_id: JORFTEXT000047340945
row:5907
article_id: JORFTEXT000047340975
row:5908
article_id: JORFTEXT000047340988
row:5909
article_id: JORFTEXT000047341043
row:5910
article_id: JORFTEXT000047341047
row:5911
article_id: JORFTEXT000047341050
row:5912
article_id: JORFTEXT000047341074
row:5913
article_id: JORFTEXT000047341078
row:5914
article_id: JORFTEXT000047341081
row:5915
article_id: JORFTEXT000047341111
row:5916
article_id: JORFTEXT000047341119
row:5917
article_id: JORFTEXT000047341125
row:5918
article_id: JORFTEXT000047341133
row:5919
article_id: JORFTEXT000047341142
row:5920
article_id: JORFTEXT000047341155
row:5921
article_id: JORFTEXT000047341167
row:5922
article_id: JORFTEXT000047341176
row:5923
article_id: JORFTEXT000047341183
row:5924
article_id: JORFTEXT000047341193
row:5925
article_id: JORFTEXT00004

row:6098
article_id: JORFTEXT000047335433
row:6099
article_id: JORFTEXT000047335436
row:6100
article_id: JORFTEXT000047335439
row:6101
article_id: JORFTEXT000047335444
row:6102
article_id: JORFTEXT000047335446
row:6103
article_id: JORFTEXT000047335448
row:6104
article_id: JORFTEXT000047335450
row:6105
article_id: JORFTEXT000047335453
row:6106
article_id: JORFTEXT000047335455
row:6107
article_id: JORFTEXT000047335457
row:6108
article_id: JORFTEXT000047335459
row:6109
article_id: JORFTEXT000047335461
row:6110
article_id: JORFTEXT000047335466
row:6111
article_id: JORFTEXT000047335473
row:6112
article_id: JORFTEXT000047335481
row:6113
article_id: JORFTEXT000047335501
row:6114
article_id: JORFTEXT000047335521
row:6115
article_id: JORFTEXT000047335526
row:6116
article_id: JORFTEXT000047335531
row:6117
article_id: JORFTEXT000047335536
row:6118
article_id: JORFTEXT000047335541
row:6119
article_id: JORFTEXT000047335549
row:6120
article_id: JORFTEXT000047335552
row:6121
article_id: JORFTEXT00004

row:6294
article_id: JORFTEXT000047324735
row:6295
article_id: JORFTEXT000047324747
row:6296
article_id: JORFTEXT000047324759
row:6297
article_id: JORFTEXT000047324763
row:6298
article_id: JORFTEXT000047324796
row:6299
article_id: JORFTEXT000047324817
row:6300
article_id: JORFTEXT000047324823
row:6301
article_id: JORFTEXT000047324873
row:6302
article_id: JORFTEXT000047324909
row:6303
article_id: JORFTEXT000047324946
row:6304
article_id: JORFTEXT000047324975
row:6305
article_id: JORFTEXT000047325004
row:6306
article_id: JORFTEXT000047325015
row:6307
article_id: JORFTEXT000047325029
row:6308
article_id: JORFTEXT000047325049
row:6309
article_id: JORFTEXT000047325055
row:6310
article_id: JORFTEXT000047325067
row:6311
article_id: JORFTEXT000047325085
row:6312
article_id: JORFTEXT000047325094
row:6313
article_id: JORFTEXT000047325108
row:6314
article_id: JORFTEXT000047325123
row:6315
article_id: JORFTEXT000047325129
row:6316
article_id: JORFTEXT000047325132
row:6317
article_id: JORFTEXT00004

row:6490
article_id: JORFTEXT000047317386
row:6491
article_id: JORFTEXT000047317519
row:6492
article_id: JORFTEXT000047317523
row:6493
article_id: JORFTEXT000047317655
row:6494
article_id: JORFTEXT000047317778
row:6495
article_id: JORFTEXT000047317909
row:6496
article_id: JORFTEXT000047317916
row:6497
article_id: JORFTEXT000047317930
row:6498
article_id: JORFTEXT000047317932
row:6499
article_id: JORFTEXT000047317934
row:6500
article_id: JORFTEXT000047317936
row:6501
article_id: JORFTEXT000047317939
row:6502
article_id: JORFTEXT000047317941
row:6503
article_id: JORFTEXT000047317943
row:6504
article_id: JORFTEXT000047317947
row:6505
article_id: JORFTEXT000047317952
row:6506
article_id: JORFTEXT000047317968
row:6507
article_id: JORFTEXT000047317986
row:6508
article_id: JORFTEXT000047318007
row:6509
article_id: JORFTEXT000047318026
row:6510
article_id: JORFTEXT000047318033
row:6511
article_id: JORFTEXT000047318036


In [39]:
print(len(docs))
print(docs[2071])
df3

12217
['JORFTEXT000047584523', 'Article 1', 'https://www.legifrance.gouv.fr/jorf/article_jo/JORFARTI000047584527', "Les modalités et conditions d'attribution de la prime de compétences spécifiques prévue par le décret du 24 mai 2023 susvisé et des taux afférents sont précisées en annexes 1 à 14."]


,article_id,child_title,child_link,child_text
0,JORFTEXT000047664400,Article 1,https://www.legifrance.gouv.fr/jorf/article_jo...,L'acquisition par l'Agence française de dévelo...
1,JORFTEXT000047664400,Article 2,https://www.legifrance.gouv.fr/jorf/article_jo...,Le présent arrêté sera publié au Journal offic...
2,JORFTEXT000047664405,Article 1,https://www.legifrance.gouv.fr/jorf/article_jo...,La cession par l'Agence française de développe...
3,JORFTEXT000047664405,Article 2,https://www.legifrance.gouv.fr/jorf/article_jo...,Le présent arrêté sera publié au Journal offic...
4,JORFTEXT000047664410,Article 1,https://www.legifrance.gouv.fr/jorf/article_jo...,Il est créé une nouvelle ligne d'obligations a...
...,...,...,...,...
12212,JORFTEXT000047317916,Article 3,https://www.legifrance.gouv.fr/jorf/article_jo...,Délégation de signature est donnée dans les li...
12213,JORFTEXT000047317916,Article 4,https://www.legifrance.gouv.fr/jorf/article_jo...,Délégation de signature est donnée dans les li...
12214,JORFTEXT000047317916,Article 5,https://www.legifrance.gouv.fr/jorf/article_jo...,Délégation de signature est donnée dans les li...
12215,JORFTEXT000047317916,Article 6,https://www.legifrance.gouv.fr/jorf/article_jo...,Délégation permanente est donnée à Mme Sandrin...


In [38]:
#merge layer 1 and layer 2+3
df_full = pd.merge(data_2, df3, on='article_id')
df_full

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## 2) Prepare data - updates

In [41]:
#!pip install tld
import pandas as pd
import json
from tld import get_tld
import re
import locale
import calendar

In [42]:
df = df_full
df.to_parquet("new.parquet")

In [43]:
#Create ID column (required for the indexing in Azure Congnitive Search)
df['ID'] = df.reset_index().index.astype(str)

In [44]:
#Create country column
#function to extract the TLD from a URL
def extract_tld(url):
    tld = get_tld(url)
    match = re.search(r'\b((?<=\.)[^.]{2}$)\b', tld)
    if match:
        return match.group(1).upper()
    else:
        return 'unknown'

# create a new 'Country' column containing the TLD (gouv.fr or co.uk => FR, UK)
df['country'] = df['article_link'].apply(extract_tld)

In [45]:
#Create year, month, day columns
#1. Find year
df['year'] = df['journal_date'].str.extract(r'\b(\d{4})\b')
#2. Find month
df['month'] = df['journal_date'].str.extract(r'\b([a-žA-Z]+)\b')
#2. Find day
df['day'] = df['journal_date'].str.extract(r'\b(^\d{1,2})\b')

In [46]:
LOCALE = 'fr_FR'
locale.setlocale(locale.LC_TIME, LOCALE)

# create a function to convert French month names to month numbers
def french_month_to_number(month):
    return list(calendar.month_name).index(month)

# apply the function to the 'month' column
df['month'] = df['month'].apply(french_month_to_number)

# convert the year, month, and day columns to a single datetime column
df['journal_dateType'] = pd.to_datetime(df[['year', 'month', 'day']]).astype(str)

# drop the original year, month, and day columns
df = df.drop(['journal_date','year', 'month', 'day'], axis=1)

In [47]:
#Reorder columns
df = df.reindex(columns=['ID','country','journal_dateType', 'journal_link', 'journal_pdf_link', 'article_id', 'article_title', 'article_link','child_title','child_link','child_text' ])

In [48]:
df

,ID,country,journal_dateType,journal_link,journal_pdf_link,article_id,article_title,article_link,child_title,child_link,child_text
0,0,FR,2023-06-11,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...,JORFTEXT000047664400,Arrêté du 25 mai 2023 portant approbation d'un...,https://www.legifrance.gouv.fr/jorf/id/JORFTEX...,Article 1,https://www.legifrance.gouv.fr/jorf/article_jo...,L'acquisition par l'Agence française de dévelo...
1,1,FR,2023-06-11,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...,JORFTEXT000047664400,Arrêté du 25 mai 2023 portant approbation d'un...,https://www.legifrance.gouv.fr/jorf/id/JORFTEX...,Article 2,https://www.legifrance.gouv.fr/jorf/article_jo...,Le présent arrêté sera publié au Journal offic...
2,2,FR,2023-06-11,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...,JORFTEXT000047664405,Arrêté du 25 mai 2023 portant approbation d'un...,https://www.legifrance.gouv.fr/jorf/id/JORFTEX...,Article 1,https://www.legifrance.gouv.fr/jorf/article_jo...,La cession par l'Agence française de développe...
3,3,FR,2023-06-11,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...,JORFTEXT000047664405,Arrêté du 25 mai 2023 portant approbation d'un...,https://www.legifrance.gouv.fr/jorf/id/JORFTEX...,Article 2,https://www.legifrance.gouv.fr/jorf/article_jo...,Le présent arrêté sera publié au Journal offic...
4,4,FR,2023-06-11,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...,JORFTEXT000047664410,Arrêté du 6 juin 2023 relatif à la création d'...,https://www.legifrance.gouv.fr/jorf/id/JORFTEX...,Article 1,https://www.legifrance.gouv.fr/jorf/article_jo...,Il est créé une nouvelle ligne d'obligations a...
...,...,...,...,...,...,...,...,...,...,...,...
12212,12212,FR,2023-03-18,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...,JORFTEXT000047317916,Décision n° 2842 du 15 mars 2023 portant délég...,https://www.legifrance.gouv.fr/jorf/id/JORFTEX...,Article 3,https://www.legifrance.gouv.fr/jorf/article_jo...,Délégation de signature est donnée dans les li...
12213,12213,FR,2023-03-18,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...,JORFTEXT000047317916,Décision n° 2842 du 15 mars 2023 portant délég...,https://www.legifrance.gouv.fr/jorf/id/JORFTEX...,Article 4,https://www.legifrance.gouv.fr/jorf/article_jo...,Délégation de signature est donnée dans les li...
12214,12214,FR,2023-03-18,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...,JORFTEXT000047317916,Décision n° 2842 du 15 mars 2023 portant délég...,https://www.legifrance.gouv.fr/jorf/id/JORFTEX...,Article 5,https://www.legifrance.gouv.fr/jorf/article_jo...,Délégation de signature est donnée dans les li...
12215,12215,FR,2023-03-18,https://www.legifrance.gouv.fr/jorf/jo/id/JORF...,https://www.legifrance.gouv.fr/download/pdf?id...,JORFTEXT000047317916,Décision n° 2842 du 15 mars 2023 portant délég...,https://www.legifrance.gouv.fr/jorf/id/JORFTEX...,Article 6,https://www.legifrance.gouv.fr/jorf/article_jo...,Délégation permanente est donnée à Mme Sandrin...


In [50]:
df.to_parquet(file_name)

## Redifine the ID key THEN load new documents into the existing index

In [87]:
df_origin = pd.read_parquet("France/data_fr_origin_prepared.parquet")

last_row_index = len(df_origin) - 1
last_row_id = df_origin.iloc[last_row_index]['ID']
print(last_row_id)

start_index_value = int(last_row_id) + 1

df_update = pd.read_parquet(file_name)

df_update['ModifiedIndex'] = df_update.index + int(start_index_value)
df_update['ID'] = df_update['ModifiedIndex'].astype(str)
df_update = df_update.drop('ModifiedIndex', axis=1)


df_update
df_update.to_parquet("happenings_from_2023-3-18_to_2023-06-12.parquet")

1021222


## Load: Injection to the current index for France in Azure Cognitive Search

In [85]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField
)

In [86]:
service_name = "pwc-leda"
admin_key = "EHBOobTZj5okalBknZsF9E2VvwAe0CPADf5k3zAaWoAzSeCEa880"


index_name = "fr-origin-1m-filterable"

# Create an SDK client
endpoint = "https://{}.search.windows.net/".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

In [88]:
df = pd.read_parquet("happenings_from_2023-3-18_to_2023-06-12.parquet")

try:
        result = df.to_json(orient="records")
        documents = json.loads(result) 
        result = search_client.upload_documents(documents=documents)
        print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
        print (ex.message)

Upload of new document succeeded: True


In [90]:
documents[1]

{'ID': '1021224',
 'country': 'FR',
 'journal_dateType': '2023-06-11',
 'journal_link': 'https://www.legifrance.gouv.fr/jorf/jo/id/JORFCONT000047664395',
 'journal_pdf_link': 'https://www.legifrance.gouv.fr/download/pdf?id=kONM7XCFtUP-8qj-5T_gzZ9zMW9r0VCLrkV8AmAAT3o=',
 'article_id': 'JORFTEXT000047664400',
 'article_title': "Arrêté du 25 mai 2023 portant approbation d'une prise de participation financière par l'Agence française de développement",
 'article_link': 'https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000047664400',
 'child_title': 'Article 2',
 'child_link': 'https://www.legifrance.gouv.fr/jorf/article_jo/JORFARTI000047664404',
 'child_text': 'Le présent arrêté sera publié au Journal officiel de la République française.'}

In [ ]:
#test

In [ ]:
service_name = "pwc-leda"
admin_key = "EHBOobTZj5okalBknZsF9E2VvwAe0CPADf5k3zAaWoAzSeCEa880"

index_name = "leda-origin"

# Create an SDK client
endpoint = "https://{}.search.windows.net/".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

In [ ]:
#Based on merged df
#Structure: journal_date,journal_link,journal_pdf_link,article_id,article_title,article_link,child_title,child_link,child_text

# Specify the index schema
name = index_name
fields = [
    SimpleField(name="ID", type=SearchFieldDataType.String, sortable=True, key=True),
    SimpleField(name="country", type=SearchFieldDataType.String, filterable=True, sortable=True),
    SimpleField(name="journal_dateType", type=SearchFieldDataType.DateTimeOffset, filterable=True, sortable=True),
    SimpleField(name="journal_link", type=SearchFieldDataType.String),
    SimpleField(name="journal_pdf_link", type=SearchFieldDataType.String),
    SimpleField(name="article_id", type=SearchFieldDataType.String, filterable=True, sortable=True),
    SearchableField(name="article_title", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
    SearchableField(name="article_link", type=SearchFieldDataType.String),
    SearchableField(name="child_title", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
    SimpleField(name="child_link", type=SearchFieldDataType.String, sortable=True), 
    SearchableField(name="child_text", type=SearchFieldDataType.String, analyzer_name="fr.lucene")
    ]
cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
scoring_profiles = []
suggester = [{'name': 'sg', 'source_fields': ['article_id']}]

In [ ]:
index = SearchIndex(
    name=name,
    fields=fields,
    scoring_profiles=scoring_profiles,
    suggesters = suggester,
    cors_options=cors_options)

try:
    result = admin_client.create_index(index)
    print ('Index', result.name, 'created')
except Exception as ex:
    print (ex)

In [ ]:
#end test

## Bonus: experimentation with Synapse - Load this new data into my dataLake

In [60]:
# install the following package 
!pip install azure-storage-file-datalake 
from azure.storage.filedatalake import DataLakeServiceClient


     ------------------------------------ 245.7/245.7 kB 885.3 kB/s eta 0:00:00
     -------------------------------------- 387.8/387.8 kB 2.4 MB/s eta 0:00:00


[https://stackoverflow.com/questions/72692155/how-can-i-upload-a-parquet-file-from-my-local-machine-to-azure-storage-data-lak]
[https://learn.microsoft.com/en-us/python/api/overview/azure/storage-file-datalake-readme?view=azure-python#uploading-a-file]

In [85]:

# Get the below details from your storage account
storage_account_name = "storageaccountbis"
storage_account_key = "uZwlPTxydwRdGQjOUR/QFionO7a1tFyZqytx4/BLJb+9heDJBpyxNJytcvXMy07qtII4QxLhdQSd+AStVdjzRA=="
container_name = "storagecontainerbis"
directory_name = "data"
file_name = "new.parquet"
#file_name = 'happenings_from_'+str(starting_update_on.year) +'-'+ str(starting_update_on.day) + '-' + str(starting_update_on.day) + '_to_'+str(end)+'.parquet'

service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
        "https", storage_account_name), credential=storage_account_key)

file_system_client = service_client.get_file_system_client(file_system=container_name)
dir_client = file_system_client.get_directory_client(directory_name)

#dir_client.create_directory()
#dir_client.open_directory("data")

local_file = pd.read_parquet(file_name)
df = pd.DataFrame(local_file).to_parquet()

file_client = dir_client.create_file(file_name)
file_client.upload_data(data=df,overwrite=True) #Either of the lines works
file_client.append_data(data=df, offset=0, length=len(df)) 

file_client.flush_data(len(df))

#file_client = dir_client.create_file("sampledata.txt")
#file_client.append_data(data, 0, len(data))
#file_client.flush_data(len(data))



{'date': datetime.datetime(2023, 3, 30, 7, 56, 35, tzinfo=datetime.timezone.utc),
 'etag': '"0x8DB30F449362BCC"',
 'last_modified': datetime.datetime(2023, 3, 30, 7, 56, 36, tzinfo=datetime.timezone.utc),
 'content_length': 0,
 'client_request_id': '64f919bb-ced0-11ed-b749-84fdd15dc446',
 'request_id': 'e41401f8-f01f-003c-35dd-6218ce000000',
 'version': '2021-12-02',
 'request_server_encrypted': False,
 'encryption_key_sha256': None,
 'lease_renewed': None}

In [ ]:
storage_account_name='<ACCOUNT_NAME>'
storage_account_key='ACCOUNT_KEY'

service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=storage_account_key)
    
file_system_client = service_client.get_file_system_client(file_system="container")

directory_client = file_system_client.get_directory_client(directory="directory")
        
file_client = directory_client.create_file("uploaded-file.parquet")

local_file = pd.read_parquet("<YOUR_FILE_NAME>.parquet")
df = pd.DataFrame(local_file).to_parquet()

file_client.upload_data(data=df,overwrite=True) #Either of the lines works
#file_client.append_data(data=df, offset=0, length=len(df)) 
file_client.flush_data(len(df))

Load updates data into index. (it will happen inside the Azure Synapse and the updateIndex will be triggered as soon as a file is load)